# **Start Section:**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Uninstall existing scikit-learn to avoid conflicts
!pip uninstall -y scikit-learn
# Install specific versions of libraries to avoid conflicts
!pip install scikit-learn==1.5.2
!pip install bayesian-optimization
!pip install optuna
!pip install gpboost
!pip install shap
!pip install ngboost
!pip install dask[dataframe]
!pip install torch seaborn
!pip install lightgbm
!pip install xgboost
!pip install lime
!pip install interpret
!pip install optunahub
!pip install cmaes
!pip install plotly kaleido
!pip install openpyxl
!pip install -U kaleido
!pip install properscoring
!pip install XlsxWriter
!pip install cython
!pip install pgbm
!pip install torch
!pip install cp
!pip install mapie
!pip install torch skorch puncc
# Reinstall scikit-learn to the version required by ngboost
!pip uninstall -y scikit-learn
!pip install scikit-learn==1.6.1
# Reinstall numpy first
!pip install numpy==1.26.4  # Use the version compatible with catboost
# Reinstall catboost
!pip install catboost

Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
  Using cached scikit_learn-1.5.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
Using cached scikit_learn-1.5.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ngboost 0.5.6 requires scikit-learn<2.0,>=1.6, but you have scikit-learn 1.5.2 which is incompatible.
  Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.5 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.2
    Uninstalling scikit-learn-1.5.2:
      Successfully uninstalled scikit-learn-1.5.

In [ ]:
# Restart the runtime to apply changes
import os
os._exit(00)

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ngboost
from scipy.stats import randint
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from gpboost import GPBoostRegressor
from ngboost import NGBRegressor
import optuna
import optunahub
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from interpret import show
from interpret.blackbox import LimeTabular, ShapKernel
from optuna.samplers import RandomSampler
import random
import time
from ngboost.distns import Normal
from ngboost.scores import LogScore
from scipy.stats import norm
from optuna.samplers import BaseSampler
from optuna.samplers import GridSampler
from optuna.samplers import TPESampler
from optuna.samplers import PartialFixedSampler
from optuna.samplers import CmaEsSampler
from optuna.samplers import QMCSampler
from optuna.samplers import NSGAIIISampler
from optuna.samplers import NSGAIISampler
from optuna.samplers import BruteForceSampler
from optuna.samplers import GPSampler
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
from interpret.glassbox import ExplainableBoostingRegressor
from interpret import show
import plotly.express as px
from io import BytesIO
from openpyxl import Workbook, load_workbook
import os
import properscoring as ps
import io
from openpyxl.drawing.image import Image as openpyxlImage
import warnings
import xlsxwriter
from openpyxl.drawing.image import Image
from pgbm.sklearn import HistGradientBoostingRegressor
import torch
from pgbm.torch import PGBM
import plotly.graph_objects as go
warnings.filterwarnings('ignore')
import pickle
import json
from mapie.subsample import Subsample
# from mapie.regression import MapieRegressor
from deel.puncc.metrics import regression_sharpness, regression_mean_coverage
from deel.puncc.api.prediction import BasePredictor, DualPredictor
from deel.puncc.regression import SplitCP, CVPlus, CQR
from deel.puncc.plotting import plot_prediction_intervals
from sklearn.model_selection import train_test_split
from typing_extensions import TypedDict
from typing import Union
# from mapie.metrics import regression_coverage_score
from sklearn.model_selection import KFold
from PIL import Image as PImage

In [4]:
train_data_path = "./drive/MyDrive/WATER QUALITY/WQI_uncertainity/train.csv"
test_data_path = "./drive/MyDrive/WATER QUALITY/WQI_uncertainity/test.csv"
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)
print("Training data loaded successfully.")
print("Test data loaded successfully.")

Training data loaded successfully.
Test data loaded successfully.


In [5]:
print("\nShape of training data:", train_data.shape)
print("First 5 rows of training data:\n", train_data.head(5))
print("\nShape of test data:", test_data.shape)
print("First 5 rows of test data:\n", test_data.head(5))


Shape of training data: (190, 152)
First 5 rows of training data:
        R1      R2      R3      R4      R5      R6      R7      R8      R9  \
0  3.5960  3.6080  3.0100  3.6130  3.3760  2.2780  1.3410  3.4570  3.3540   
1  2.9540  3.0470  3.1160  4.2640  4.4090  3.4810  2.3760  3.4030  3.4320   
2  2.4870  2.9430  2.5290  3.7280  2.9870  1.8890  1.9960  2.1020  1.6180   
3  1.3640  1.4186  1.3789  1.4776  1.5063  1.4047  1.2648  1.5166  1.5443   
4  1.3074  1.3820  1.3476  1.3986  1.3403  1.3004  1.3719  1.3984  1.3865   

      R10  ...    R143    R144    R145    R146    R147   R148    R149    R150  \
0  1.9900  ...  2.7930  1.9660  2.3070  1.5280  1.9380  3.371  2.2060  1.5220   
1  2.6350  ...  3.0260  2.3530  2.0400  1.7400  2.6250  4.026  2.9010  1.7450   
2  1.2720  ...  1.2760  1.5990  1.8610  1.0290  1.8690  3.905  2.3200  1.3310   
3  1.3834  ...  1.2223  1.0919  1.0995  1.0856  1.1728  1.315  1.2341  1.0753   
4  1.3964  ...  1.1691  1.0689  1.1230  1.1384  1.1810  1.235  1

In [6]:
X_train = train_data.iloc[:, :-1]
y_train = train_data.iloc[:, -1]
X_test = test_data.iloc[:, :-1]
y_test = test_data.iloc[:, -1]
x_test= X_test
print("\nShape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)


Shape of X_train: (190, 151)
Shape of y_train: (190,)
Shape of X_test: (82, 151)
Shape of y_test: (82,)


In [7]:
# Apply z-score normalization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Print the first five rows of the normalized data
print("\nFirst five rows of normalized X_train:")
print(X_train[:5])

print("\nFirst five rows of normalized X_test:")
print(X_test[:5])


First five rows of normalized X_train:
[[ 1.22103295  1.13089815  0.61081314  0.73682842  0.54673262 -0.0261968
  -0.51531151  0.70303503  0.5510272  -0.2441054   0.17648144  0.69219002
   0.9485328   0.82481901  0.34566436 -0.40609522 -0.463771    0.43872889
   0.89421381  1.05057715  1.04529849  1.03900179  0.42575888  0.173456
   0.45642529  0.83037026  0.89010153  0.77972274  0.71866378  0.6206252
   0.82167079  0.97412629  0.67298061  0.39758814  0.52679374  0.94375937
   0.74197071  0.86577935  1.27064247  1.51437415  0.46596778 -0.30888056
  -0.12461695  0.49591603  0.72957494  0.5811918   0.31848773  0.53466849
   0.20246884  0.11900459  0.99635035  0.72105768  0.43065398  0.36777539
   0.10638109 -0.0263905   0.2318847   0.82337505  0.74363607  0.85248236
   0.71669043 -0.11909161 -0.10843386  0.51688952  0.84551287 -0.1576873
  -0.12716017  0.37884167  0.35396679 -0.24556803  0.07962918  0.82628894
   0.54494889  1.02963853  1.17379609  0.60350175 -0.17692428 -0.31350223
   

# **Functions:**

In [8]:
feature_names = [f'R{i}' for i in range(1, 152)]

In [10]:
def get_best_model_params(results, model_name):
    # Map model names to dictionary keys, assuming keys are strings like 'XGBoost' and not objects
    model_keys = {
        'LightGBM': 'LightGBM',
        'XGBoost': 'XGBoost',
        'GPBoost': 'GPBoost',
        'NGBoost': 'NGBoost',
        'HGBR' : 'HistGradientBoosting',
        'PGBM' : 'PGBM'
    }

    # Ensure the requested model name is valid
    if model_name not in model_keys:
        raise ValueError(f"Model name '{model_name}' is not recognized. Available models are: {list(model_keys.keys())}")

    # Filter out entries for the specified model
    model_entries = {key: value for key, value in results.items() if key[0] == model_keys[model_name]}

    # Find the entry with the best (lowest) 'best_score'
    best_entry_key, best_entry_value = min(model_entries.items(), key=lambda item: item[1]['best_score'])

    # Return the best hyperparameters
    return best_entry_value['best_params']

In [11]:
def generate_and_save_plots(predictions_df, y_test, excel_path):
    # 1. Predicted vs. Actual Plot
    plt.figure(figsize=(12, 8))
    sns.scatterplot(x='Actual', y='Mean', data=predictions_df)
    plt.plot([predictions_df['Actual'].min(), predictions_df['Actual'].max()],
             [predictions_df['Actual'].min(), predictions_df['Actual'].max()],
             color='red', linestyle='--')
    plt.title('Predicted WQI vs. Actual WQI')
    plt.xlabel('Actual WQI')
    plt.ylabel('Predicted Mean WQI')
    plt.tight_layout()
    predicted_vs_actual_path = 'predicted_vs_actual.png'
    plt.savefig(predicted_vs_actual_path)
    plt.close()

    # 2. Uncertainty Bands
    plt.figure(figsize=(12, 8))
    plt.errorbar(x=range(len(predictions_df)), y=predictions_df['Mean'],
                 yerr=predictions_df['StdDev'], fmt='o', markersize=4, ecolor='red', elinewidth=1, capsize=5, alpha=0.7)
    plt.plot(range(len(predictions_df)), predictions_df['Mean'], color='black', linewidth=0.5, alpha=0.7)
    plt.title('Predicted Mean with Uncertainty Bands', fontsize=16)
    plt.xlabel('Sample Number', fontsize=14)
    plt.ylabel('Predicted Mean WQI')
    plt.tight_layout()
    uncertainty_bands_path = 'uncertainty_bands.png'
    plt.savefig(uncertainty_bands_path)
    plt.close()

    # 3. Histogram of Standard Deviations
    plt.figure(figsize=(12, 8))
    sns.histplot(predictions_df['StdDev'], bins=20, kde=True)
    plt.title('Distribution of Predicted Standard Deviations')
    plt.xlabel('Standard Deviation')
    plt.ylabel('Frequency')
    plt.tight_layout()
    histogram_stddev_path = 'histogram_stddev.png'
    plt.savefig(histogram_stddev_path)
    plt.close()

    # 4. Residual Plot
    residuals = predictions_df['Actual'] - predictions_df['Mean']
    plt.figure(figsize=(12, 8))
    sns.scatterplot(x=predictions_df['Mean'], y=residuals)
    plt.axhline(0, color='red', linestyle='--')
    plt.title('Residual Plot')
    plt.xlabel('Predicted Mean WQI')
    plt.ylabel('Residuals')
    plt.tight_layout()
    residual_plot_path = 'residual_plot.png'
    plt.savefig(residual_plot_path)
    plt.close()

    # Plot the standard deviation of the predictions
    plt.figure(figsize=(12, 6))
    plt.plot(
        predictions_df['StdDev'],
        label='Standard Deviation',
        color='green',
        marker='o',
        linestyle='dashed'
    )
    plt.xlabel('Sample Number')
    plt.ylabel('Standard Deviation')
    plt.title('Standard Deviation of Predictions')
    plt.legend(loc='upper left', fontsize='small', bbox_to_anchor=(1.05, 1), borderaxespad=0.)
    plt.tight_layout()
    stddev_plot_path = 'stddev_plot.png'
    plt.savefig(stddev_plot_path)
    plt.close()

    # Plot the actual vs predicted mean with confidence intervals
    plt.figure(figsize=(12, 6))
    plt.plot(
        y_test,
        label='Actual WQI',
        color='black',
        marker='o',
        linestyle='dashed',
        linewidth=1,
        markersize=4
    )
    plt.plot(
        predictions_df['Mean'],
        label='Predicted Mean WQI',
        color='red',
        marker='x',
        linestyle='dashed',
        linewidth=1,
        markersize=4
    )
    confidence_interval = 1.96 * predictions_df['StdDev']
    plt.fill_between(
        np.arange(len(predictions_df)),
        predictions_df['Mean'] - confidence_interval,
        predictions_df['Mean'] + confidence_interval,
        color='blue',
        alpha=0.2,
        label='95% Confidence Interval'
    )
    plt.xlabel('Sample Number')
    plt.ylabel('WQI')
    plt.title('Predictions with Confidence Intervals')
    plt.legend(loc='upper left', fontsize='small', bbox_to_anchor=(1.05, 1), borderaxespad=0.)
    plt.tight_layout()
    prediction_plot_path = 'prediction_plot.png'
    plt.savefig(prediction_plot_path)
    plt.close()

    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(excel_path, engine='xlsxwriter')

    # Write the dataframe to a worksheet.
    predictions_df.to_excel(writer, sheet_name='Predictions', index=False)

    # Access the XlsxWriter workbook and worksheet objects from the writer object.
    workbook = writer.book

    # Insert the plots into new worksheets.
    worksheet_predicted_vs_actual = workbook.add_worksheet('Predicted vs Actual')
    worksheet_predicted_vs_actual.insert_image('A1', predicted_vs_actual_path)

    worksheet_uncertainty_bands = workbook.add_worksheet('Uncertainty Bands')
    worksheet_uncertainty_bands.insert_image('A1', uncertainty_bands_path)

    worksheet_histogram_stddev = workbook.add_worksheet('Histogram StdDev')
    worksheet_histogram_stddev.insert_image('A1', histogram_stddev_path)

    worksheet_residual_plot = workbook.add_worksheet('Residual Plot')
    worksheet_residual_plot.insert_image('A1', residual_plot_path)

    worksheet_stddev = workbook.add_worksheet('StdDev Plot')
    worksheet_stddev.insert_image('A1', stddev_plot_path)

    worksheet_prediction = workbook.add_worksheet('Predictions Plot')
    worksheet_prediction.insert_image('A1', prediction_plot_path)

    # Close the Pandas Excel writer and output the Excel file.
    writer.close()
    print(f"Excel file saved at {excel_path}")

    # Optionally, remove the image files now that they're embedded in the Excel file
    os.remove(predicted_vs_actual_path)
    os.remove(uncertainty_bands_path)
    os.remove(histogram_stddev_path)
    os.remove(residual_plot_path)
    os.remove(stddev_plot_path)
    os.remove(prediction_plot_path)

In [12]:
# Define the path to the folder
folder_path = "/content/drive/MyDrive/ml_concrete"

# Create the folder if it doesn't exist
os.makedirs(folder_path, exist_ok=True)

def plot_metrics_and_save_to_excel(writer, predictions_df, y_true, model_name):
    # Calculate metrics
    mean_pred = predictions_df['Mean']
    std_pred = predictions_df['StdDev']

    # Sharpness is the standard deviation
    sharpness = std_pred.mean()

    # CRPS for each prediction
    crps_values = ps.crps_gaussian(y_true, mean_pred, std_pred)

    # Negative Log Likelihood (NLL) for each prediction
    nll_values = -norm.logpdf(y_true, loc=mean_pred, scale=std_pred)
    nll = nll_values.mean()

    # Calibration (Validity)
    percentiles = np.linspace(0, 100, 101)
    calibration = [np.mean((y_true <= np.percentile(mean_pred + std_pred * np.random.randn(*mean_pred.shape), p))) for p in percentiles]

    # Create a DataFrame to store the metrics
    metrics_df = pd.DataFrame({
        'Mean Prediction': mean_pred,
        'Standard Deviation': std_pred,
        'CRPS': crps_values,
        'NLL': nll_values
    })

    # Add summary statistics
    summary_df = pd.DataFrame({
        'Metric': ['Sharpness', 'Mean CRPS', 'Mean NLL'],
        'Value': [sharpness, crps_values.mean(), nll]
    })

    # Write the metrics and summary to the Excel sheet
    metrics_sheet_name = f'{model_name} Metrics'
    metrics_df.to_excel(writer, sheet_name=metrics_sheet_name, index=False)

    summary_sheet_name = f'{model_name} Summary'
    summary_df.to_excel(writer, sheet_name=summary_sheet_name, index=False)

    # Plotting
    fig, axs = plt.subplots(2, 2, figsize=(12, 10))

    # Sharpness plot
    axs[0, 0].hist(std_pred, bins=30, color='skyblue', edgecolor='black')
    axs[0, 0].set_title(f'Sharpness (StdDev) for {model_name}')
    axs[0, 0].set_xlabel('Standard Deviation')
    axs[0, 0].set_ylabel('Frequency')

    # CRPS plot (distribution)
    axs[0, 1].hist(crps_values, bins=30, color='lightgreen', edgecolor='black')
    axs[0, 1].set_title(f'CRPS Distribution for {model_name}')
    axs[0, 1].set_xlabel('CRPS')
    axs[0, 1].set_ylabel('Frequency')

    # NLL plot (distribution)
    axs[1, 0].hist(nll_values, bins=30, color='salmon', edgecolor='black')
    axs[1, 0].set_title(f'NLL Distribution for {model_name}')
    axs[1, 0].set_xlabel('NLL')
    axs[1, 0].set_ylabel('Frequency')

    # Calibration plot
    axs[1, 1].plot(percentiles, calibration, label='Calibration Curve')
    axs[1, 1].plot(percentiles, percentiles / 100, linestyle='--', label='Perfect Calibration')
    axs[1, 1].set_title(f'Calibration (Validity) for {model_name}')
    axs[1, 1].set_xlabel('Predicted Percentile')
    axs[1, 1].set_ylabel('Observed Frequency')
    axs[1, 1].legend()

    plt.tight_layout()

    # Save plots to images
    plot_image_path = f'{model_name}_plots.png'
    fig.savefig(plot_image_path)
    plt.close(fig)

    # Insert the plot image into the Excel sheet
    sheet_name = f'{model_name} Plots'
    if sheet_name not in writer.sheets:  # Check if sheet already exists
        # If sheet doesn't exist, create a new one
        worksheet = writer.book.add_worksheet(sheet_name)
    else:
        # If sheet exists, get the existing sheet object to avoid duplicates
        worksheet = writer.sheets[sheet_name]
    worksheet.insert_image('A1', plot_image_path)

# **Hyperparameter Tuning using Autosampler Optuna**

In [13]:
best_scores_autosampler ={('Random Forest', 'MedianPruner'): {'best_score': 926.9746881386717,
  'best_params': {'n_estimators': 700,
   'criterion': 'squared_error',
   'max_depth': 20,
   'min_samples_split': 0.01,
   'min_samples_leaf': 1,
   'min_weight_fraction_leaf': 0.0,
   'max_features': 0.5,
   'max_leaf_nodes': 50,
   'min_impurity_decrease': 0.2,
   'n_jobs': -1,
   'random_state': 42,
   'verbose': 0,
   'warm_start': False,
   'ccp_alpha': 0.01},
  'test_mse': 926.9746881386717,
  'test_rmse': 30.446259017138242,
  'test_corr_coef': 0.9380539187700511,
  'pruner': 'MedianPruner'},
 ('Random Forest', 'NopPruner'): {'best_score': 937.9560207935197,
  'best_params': {'n_estimators': 700,
   'criterion': 'squared_error',
   'max_depth': 20,
   'min_samples_split': 2,
   'min_samples_leaf': 1,
   'min_weight_fraction_leaf': 0.0,
   'max_features': 0.3,
   'max_leaf_nodes': None,
   'min_impurity_decrease': 0.01,
   'n_jobs': -1,
   'random_state': 42,
   'verbose': 0,
   'warm_start': False,
   'ccp_alpha': 0.0},
  'test_mse': 937.9560207935197,
  'test_rmse': 30.62606766781396,
  'test_corr_coef': 0.9338201932833533,
  'pruner': 'NopPruner'},
 ('Random Forest', 'PatientPruner'): {'best_score': 928.3935142510413,
  'best_params': {'n_estimators': 300,
   'criterion': 'squared_error',
   'max_depth': 30,
   'min_samples_split': 0.01,
   'min_samples_leaf': 1,
   'min_weight_fraction_leaf': 0.0,
   'max_features': 0.3,
   'max_leaf_nodes': 50,
   'min_impurity_decrease': 0.0,
   'n_jobs': -1,
   'random_state': 42,
   'verbose': 0,
   'warm_start': False,
   'ccp_alpha': 0.0},
  'test_mse': 928.3935142510413,
  'test_rmse': 30.469550607960095,
  'test_corr_coef': 0.9318269362838447,
  'pruner': 'PatientPruner'},
 ('Random Forest', 'PercentilePruner'): {'best_score': 936.3276968392234,
  'best_params': {'n_estimators': 700,
   'criterion': 'friedman_mse',
   'max_depth': 30,
   'min_samples_split': 2,
   'min_samples_leaf': 1,
   'min_weight_fraction_leaf': 0.0,
   'max_features': 1.0,
   'max_leaf_nodes': None,
   'min_impurity_decrease': 0.2,
   'n_jobs': -1,
   'random_state': 42,
   'verbose': 0,
   'warm_start': False,
   'ccp_alpha': 0.001},
  'test_mse': 936.3276968392234,
  'test_rmse': 30.599472166023116,
  'test_corr_coef': 0.9443155093176682,
  'pruner': 'PercentilePruner'},
 ('Random Forest',
  'SuccessiveHalvingPruner'): {'best_score': 922.9224466496914, 'best_params': {'n_estimators': 300,
   'criterion': 'friedman_mse',
   'max_depth': None,
   'min_samples_split': 2,
   'min_samples_leaf': 1,
   'min_weight_fraction_leaf': 0.0,
   'max_features': 0.3,
   'max_leaf_nodes': None,
   'min_impurity_decrease': 0.2,
   'n_jobs': -1,
   'random_state': 42,
   'verbose': 0,
   'warm_start': False,
   'ccp_alpha': 0.01}, 'test_mse': 922.9224466496914, 'test_rmse': 30.379638685305185, 'test_corr_coef': 0.9344935993588843, 'pruner': 'SuccessiveHalvingPruner'},
 ('Random Forest', 'HyperbandPruner'): {'best_score': 928.6979037649045,
  'best_params': {'n_estimators': 200,
   'criterion': 'squared_error',
   'max_depth': 30,
   'min_samples_split': 2,
   'min_samples_leaf': 1,
   'min_weight_fraction_leaf': 0.0,
   'max_features': 0.3,
   'max_leaf_nodes': 50,
   'min_impurity_decrease': 0.2,
   'n_jobs': -1,
   'random_state': 42,
   'verbose': 0,
   'warm_start': False,
   'ccp_alpha': 0.1},
  'test_mse': 928.6979037649045,
  'test_rmse': 30.474545177326345,
  'test_corr_coef': 0.9305524100852461,
  'pruner': 'HyperbandPruner'},
 ('Random Forest', 'ThresholdPruner'): {'best_score': 929.8587794201471,
  'best_params': {'n_estimators': 200,
   'criterion': 'squared_error',
   'max_depth': 20,
   'min_samples_split': 2,
   'min_samples_leaf': 1,
   'min_weight_fraction_leaf': 0.0,
   'max_features': 0.3,
   'max_leaf_nodes': 100,
   'min_impurity_decrease': 0.2,
   'n_jobs': -1,
   'random_state': 42,
   'verbose': 0,
   'warm_start': False,
   'ccp_alpha': 0.01},
  'test_mse': 929.8587794201471,
  'test_rmse': 30.49358587342832,
  'test_corr_coef': 0.930542178541985,
  'pruner': 'ThresholdPruner'},
 ('Random Forest', 'WilcoxonPruner'): {'best_score': 928.8038145612356,
  'best_params': {'n_estimators': 300,
   'criterion': 'friedman_mse',
   'max_depth': 10,
   'min_samples_split': 0.01,
   'min_samples_leaf': 1,
   'min_weight_fraction_leaf': 0.0,
   'max_features': 0.3,
   'max_leaf_nodes': 50,
   'min_impurity_decrease': 0.2,
   'n_jobs': -1,
   'random_state': 42,
   'verbose': 0,
   'warm_start': False,
   'ccp_alpha': 0.05},
  'test_mse': 928.8038145612356,
  'test_rmse': 30.476282820600606,
  'test_corr_coef': 0.931846820097067,
  'pruner': 'WilcoxonPruner'},
 ('Gradient Boosting', 'MedianPruner'): {'best_score': 826.8450802541028,
  'best_params': {'loss': 'squared_error',
   'learning_rate': 0.2,
   'n_estimators': 100,
   'subsample': 0.5,
   'criterion': 'squared_error',
   'min_samples_split': 0.01,
   'min_samples_leaf': 1,
   'min_weight_fraction_leaf': 0.1,
   'max_depth': 10,
   'min_impurity_decrease': 0.0,
   'init': None,
   'random_state': 42,
   'max_features': None,
   'alpha': 0.5,
   'verbose': 0,
   'max_leaf_nodes': 30,
   'warm_start': False,
   'validation_fraction': 0.1,
   'n_iter_no_change': 10,
   'tol': 0.0001,
   'ccp_alpha': 0.0},
  'test_mse': 826.8450802541028,
  'test_rmse': 28.754914019243785,
  'test_corr_coef': 0.9279449171502246,
  'pruner': 'MedianPruner'},
 ('Gradient Boosting', 'NopPruner'): {'best_score': 781.7895087976207,
  'best_params': {'loss': 'squared_error',
   'learning_rate': 0.05,
   'n_estimators': 700,
   'subsample': 0.7,
   'criterion': 'squared_error',
   'min_samples_split': 2,
   'min_samples_leaf': 0.01,
   'min_weight_fraction_leaf': 0.01,
   'max_depth': 10,
   'min_impurity_decrease': 0.1,
   'init': None,
   'random_state': 42,
   'max_features': 'sqrt',
   'alpha': 0.5,
   'verbose': 0,
   'max_leaf_nodes': 30,
   'warm_start': False,
   'validation_fraction': 0.1,
   'n_iter_no_change': 20,
   'tol': 0.0001,
   'ccp_alpha': 0.001},
  'test_mse': 781.7895087976207,
  'test_rmse': 27.96049907990951,
  'test_corr_coef': 0.9341384925660862,
  'pruner': 'NopPruner'},
 ('Gradient Boosting', 'PatientPruner'): {'best_score': 668.5962917226813,
  'best_params': {'loss': 'squared_error',
   'learning_rate': 0.05,
   'n_estimators': 700,
   'subsample': 0.5,
   'criterion': 'squared_error',
   'min_samples_split': 2,
   'min_samples_leaf': 1,
   'min_weight_fraction_leaf': 0.0,
   'max_depth': 3,
   'min_impurity_decrease': 0.1,
   'init': None,
   'random_state': 42,
   'max_features': 'sqrt',
   'alpha': 0.9,
   'verbose': 0,
   'max_leaf_nodes': None,
   'warm_start': False,
   'validation_fraction': 0.1,
   'n_iter_no_change': 10,
   'tol': 0.001,
   'ccp_alpha': 0.01},
  'test_mse': 668.5962917226813,
  'test_rmse': 25.857229003175906,
  'test_corr_coef': 0.9420302992631479,
  'pruner': 'PatientPruner'},
 ('Gradient Boosting', 'PercentilePruner'): {'best_score': 696.7060074905164,
  'best_params': {'loss': 'huber',
   'learning_rate': 0.2,
   'n_estimators': 500,
   'subsample': 0.5,
   'criterion': 'friedman_mse',
   'min_samples_split': 0.01,
   'min_samples_leaf': 1,
   'min_weight_fraction_leaf': 0.0,
   'max_depth': 3,
   'min_impurity_decrease': 0.0,
   'init': None,
   'random_state': 42,
   'max_features': None,
   'alpha': 0.5,
   'verbose': 0,
   'max_leaf_nodes': 10,
   'warm_start': False,
   'validation_fraction': 0.1,
   'n_iter_no_change': None,
   'tol': 0.001,
   'ccp_alpha': 0.0},
  'test_mse': 696.7060074905164,
  'test_rmse': 26.395189097457067,
  'test_corr_coef': 0.946479542303648,
  'pruner': 'PercentilePruner'},
 ('Gradient Boosting',
  'SuccessiveHalvingPruner'): {'best_score': 771.7055331834227, 'best_params': {'loss': 'squared_error',
   'learning_rate': 0.05,
   'n_estimators': 300,
   'subsample': 0.5,
   'criterion': 'squared_error',
   'min_samples_split': 0.01,
   'min_samples_leaf': 1,
   'min_weight_fraction_leaf': 0.01,
   'max_depth': 10,
   'min_impurity_decrease': 0.1,
   'init': None,
   'random_state': 42,
   'max_features': 0.5,
   'alpha': 0.9,
   'verbose': 0,
   'max_leaf_nodes': None,
   'warm_start': False,
   'validation_fraction': 0.1,
   'n_iter_no_change': None,
   'tol': 0.001,
   'ccp_alpha': 0.001}, 'test_mse': 771.7055331834227, 'test_rmse': 27.779588427178375, 'test_corr_coef': 0.9441148112370387, 'pruner': 'SuccessiveHalvingPruner'},
 ('Gradient Boosting', 'HyperbandPruner'): {'best_score': 702.5066063040902,
  'best_params': {'loss': 'squared_error',
   'learning_rate': 0.05,
   'n_estimators': 500,
   'subsample': 0.5,
   'criterion': 'squared_error',
   'min_samples_split': 10,
   'min_samples_leaf': 1,
   'min_weight_fraction_leaf': 0.01,
   'max_depth': 10,
   'min_impurity_decrease': 0.0,
   'init': None,
   'random_state': 42,
   'max_features': 'sqrt',
   'alpha': 0.9,
   'verbose': 0,
   'max_leaf_nodes': 50,
   'warm_start': False,
   'validation_fraction': 0.1,
   'n_iter_no_change': None,
   'tol': 0.0001,
   'ccp_alpha': 0.01},
  'test_mse': 702.5066063040902,
  'test_rmse': 26.504841186169937,
  'test_corr_coef': 0.9410771823195679,
  'pruner': 'HyperbandPruner'},
 ('Gradient Boosting', 'ThresholdPruner'): {'best_score': 716.4136543206561,
  'best_params': {'loss': 'absolute_error',
   'learning_rate': 0.2,
   'n_estimators': 200,
   'subsample': 0.7,
   'criterion': 'friedman_mse',
   'min_samples_split': 2,
   'min_samples_leaf': 0.01,
   'min_weight_fraction_leaf': 0.0,
   'max_depth': 5,
   'min_impurity_decrease': 0.0,
   'init': None,
   'random_state': 42,
   'max_features': 0.5,
   'alpha': 0.1,
   'verbose': 0,
   'max_leaf_nodes': 30,
   'warm_start': False,
   'validation_fraction': 0.1,
   'n_iter_no_change': None,
   'tol': 0.001,
   'ccp_alpha': 0.01},
  'test_mse': 716.4136543206561,
  'test_rmse': 26.765904698340687,
  'test_corr_coef': 0.9402314074316714,
  'pruner': 'ThresholdPruner'},
 ('Gradient Boosting', 'WilcoxonPruner'): {'best_score': 727.7486334239142,
  'best_params': {'loss': 'absolute_error',
   'learning_rate': 0.1,
   'n_estimators': 500,
   'subsample': 1.0,
   'criterion': 'friedman_mse',
   'min_samples_split': 10,
   'min_samples_leaf': 1,
   'min_weight_fraction_leaf': 0.01,
   'max_depth': 5,
   'min_impurity_decrease': 0.1,
   'init': None,
   'random_state': 42,
   'max_features': 'sqrt',
   'alpha': 0.9,
   'verbose': 0,
   'max_leaf_nodes': 10,
   'warm_start': False,
   'validation_fraction': 0.1,
   'n_iter_no_change': None,
   'tol': 0.0001,
   'ccp_alpha': 0.001},
  'test_mse': 727.7486334239142,
  'test_rmse': 26.976816591731392,
  'test_corr_coef': 0.9360999910400127,
  'pruner': 'WilcoxonPruner'},
 ('XGBoost', 'MedianPruner'): {'best_score': 690.8297729492188,
  'best_params': {'n_estimators': 200,
   'learning_rate': 0.1,
   'max_depth': 3,
   'min_child_weight': 1,
   'gamma': 0.5,
   'subsample': 0.5,
   'colsample_bytree': 0.5,
   'colsample_bylevel': 0.7,
   'reg_alpha': 1,
   'reg_lambda': 1,
   'objective': 'reg:squarederror',
   'random_state': 42,
   'n_jobs': -1},
  'test_mse': 690.8297729492188,
  'test_rmse': 26.283640785652562,
  'test_corr_coef': 0.9457246603165833,
  'pruner': 'MedianPruner'},
 ('XGBoost', 'NopPruner'): {'best_score': 686.6576538085938,
  'best_params': {'n_estimators': 500,
   'learning_rate': 0.1,
   'max_depth': 5,
   'min_child_weight': 1,
   'gamma': 0.5,
   'subsample': 0.7,
   'colsample_bytree': 0.7,
   'colsample_bylevel': 0.5,
   'reg_alpha': 0.01,
   'reg_lambda': 10,
   'objective': 'reg:squarederror',
   'random_state': 42,
   'n_jobs': -1},
  'test_mse': 686.6576538085938,
  'test_rmse': 26.204153369429697,
  'test_corr_coef': 0.9497348647658967,
  'pruner': 'NopPruner'},
 ('XGBoost', 'PatientPruner'): {'best_score': 622.8858642578125,
  'best_params': {'n_estimators': 500,
   'learning_rate': 0.1,
   'max_depth': 3,
   'min_child_weight': 3,
   'gamma': 0.5,
   'subsample': 0.5,
   'colsample_bytree': 0.7,
   'colsample_bylevel': 0.9,
   'reg_alpha': 1,
   'reg_lambda': 10,
   'objective': 'reg:squarederror',
   'random_state': 42,
   'n_jobs': -1},
  'test_mse': 622.8858642578125,
  'test_rmse': 24.957681467993226,
  'test_corr_coef': 0.9526383255315664,
  'pruner': 'PatientPruner'},
 ('XGBoost', 'PercentilePruner'): {'best_score': 605.669677734375,
  'best_params': {'n_estimators': 400,
   'learning_rate': 0.15,
   'max_depth': 7,
   'min_child_weight': 3,
   'gamma': 0.1,
   'subsample': 0.6,
   'colsample_bytree': 0.7,
   'colsample_bylevel': 0.7,
   'reg_alpha': 0,
   'reg_lambda': 5,
   'objective': 'reg:squarederror',
   'random_state': 42,
   'n_jobs': -1},
  'test_mse': 605.669677734375,
  'test_rmse': 24.61035712325961,
  'test_corr_coef': 0.9508572219911771,
  'pruner': 'PercentilePruner'},
 ('XGBoost', 'SuccessiveHalvingPruner'): {'best_score': 654.443603515625,
  'best_params': {'n_estimators': 500,
   'learning_rate': 0.15,
   'max_depth': 3,
   'min_child_weight': 5,
   'gamma': 0.5,
   'subsample': 0.7,
   'colsample_bytree': 0.7,
   'colsample_bylevel': 0.7,
   'reg_alpha': 1,
   'reg_lambda': 10,
   'objective': 'reg:squarederror',
   'random_state': 42,
   'n_jobs': -1},
  'test_mse': 654.443603515625,
  'test_rmse': 25.58209536991888,
  'test_corr_coef': 0.9481971540163014,
  'pruner': 'SuccessiveHalvingPruner'},
 ('XGBoost', 'HyperbandPruner'): {'best_score': 664.0693359375,
  'best_params': {'n_estimators': 300,
   'learning_rate': 0.15,
   'max_depth': 5,
   'min_child_weight': 3,
   'gamma': 0.5,
   'subsample': 0.9,
   'colsample_bytree': 0.5,
   'colsample_bylevel': 0.5,
   'reg_alpha': 0.01,
   'reg_lambda': 5,
   'objective': 'reg:squarederror',
   'random_state': 42,
   'n_jobs': -1},
  'test_mse': 664.0693359375,
  'test_rmse': 25.769542796438976,
  'test_corr_coef': 0.9473404883534584,
  'pruner': 'HyperbandPruner'},
 ('XGBoost', 'ThresholdPruner'): {'best_score': 687.9976806640625,
  'best_params': {'n_estimators': 400,
   'learning_rate': 0.05,
   'max_depth': 3,
   'min_child_weight': 3,
   'gamma': 1,
   'subsample': 0.7,
   'colsample_bytree': 0.9,
   'colsample_bylevel': 0.5,
   'reg_alpha': 0.01,
   'reg_lambda': 5,
   'objective': 'reg:squarederror',
   'random_state': 42,
   'n_jobs': -1},
  'test_mse': 687.9976806640625,
  'test_rmse': 26.229709885243917,
  'test_corr_coef': 0.9493622590036852,
  'pruner': 'ThresholdPruner'},
 ('XGBoost', 'WilcoxonPruner'): {'best_score': 690.7450561523438,
  'best_params': {'n_estimators': 500,
   'learning_rate': 0.15,
   'max_depth': 7,
   'min_child_weight': 1,
   'gamma': 0,
   'subsample': 0.5,
   'colsample_bytree': 0.9,
   'colsample_bylevel': 0.7,
   'reg_alpha': 0,
   'reg_lambda': 5,
   'objective': 'reg:squarederror',
   'random_state': 42,
   'n_jobs': -1},
  'test_mse': 690.7450561523438,
  'test_rmse': 26.28202914830481,
  'test_corr_coef': 0.9434444285701333,
  'pruner': 'WilcoxonPruner'},
 ('LightGBM', 'MedianPruner'): {'best_score': 687.6213685241634,
  'best_params': {'n_estimators': 300,
   'learning_rate': 0.1,
   'num_leaves': 63,
   'max_depth': 3,
   'min_child_samples': 1,
   'subsample': 0.5,
   'colsample_bytree': 0.5,
   'reg_alpha': 0,
   'reg_lambda': 1,
   'min_child_weight': 0.01,
   'bagging_freq': 1,
   'objective': 'regression',
   'random_state': 42,
   'n_jobs': -1,
   'verbose': -1},
  'test_mse': 687.6213685241634,
  'test_rmse': 26.22253550906478,
  'test_corr_coef': 0.9486948872823441,
  'pruner': 'MedianPruner'},
 ('LightGBM', 'NopPruner'): {'best_score': 748.6190203392741,
  'best_params': {'n_estimators': 500,
   'learning_rate': 0.1,
   'num_leaves': 63,
   'max_depth': 3,
   'min_child_samples': 5,
   'subsample': 0.8,
   'colsample_bytree': 0.9,
   'reg_alpha': 0.1,
   'reg_lambda': 10,
   'min_child_weight': 0.001,
   'bagging_freq': 5,
   'objective': 'regression',
   'random_state': 42,
   'n_jobs': -1,
   'verbose': -1},
  'test_mse': 748.6190203392741,
  'test_rmse': 27.360903134569117,
  'test_corr_coef': 0.9450914571670231,
  'pruner': 'NopPruner'},
 ('LightGBM', 'PatientPruner'): {'best_score': 707.208125570096,
  'best_params': {'n_estimators': 400,
   'learning_rate': 0.1,
   'num_leaves': 15,
   'max_depth': -1,
   'min_child_samples': 10,
   'subsample': 0.8,
   'colsample_bytree': 0.7,
   'reg_alpha': 0.1,
   'reg_lambda': 10,
   'min_child_weight': 1e-05,
   'bagging_freq': 5,
   'objective': 'regression',
   'random_state': 42,
   'n_jobs': -1,
   'verbose': -1},
  'test_mse': 707.208125570096,
  'test_rmse': 26.59338499646286,
  'test_corr_coef': 0.9407200289406664,
  'pruner': 'PatientPruner'},
 ('LightGBM', 'PercentilePruner'): {'best_score': 766.6062716200108,
  'best_params': {'n_estimators': 400,
   'learning_rate': 0.05,
   'num_leaves': 63,
   'max_depth': 5,
   'min_child_samples': 20,
   'subsample': 0.8,
   'colsample_bytree': 0.5,
   'reg_alpha': 0.01,
   'reg_lambda': 0.1,
   'min_child_weight': 0.001,
   'bagging_freq': 0,
   'objective': 'regression',
   'random_state': 42,
   'n_jobs': -1,
   'verbose': -1},
  'test_mse': 766.6062716200108,
  'test_rmse': 27.687655581865556,
  'test_corr_coef': 0.9345182922130101,
  'pruner': 'PercentilePruner'},
 ('LightGBM', 'SuccessiveHalvingPruner'): {'best_score': 587.9233862763272,
  'best_params': {'n_estimators': 400,
   'learning_rate': 0.1,
   'num_leaves': 15,
   'max_depth': 5,
   'min_child_samples': 5,
   'subsample': 0.5,
   'colsample_bytree': 0.5,
   'reg_alpha': 0,
   'reg_lambda': 1,
   'min_child_weight': 0.001,
   'bagging_freq': 1,
   'objective': 'regression',
   'random_state': 42,
   'n_jobs': -1,
   'verbose': -1},
  'test_mse': 587.9233862763272,
  'test_rmse': 24.247131506145777,
  'test_corr_coef': 0.9577393597234097,
  'pruner': 'SuccessiveHalvingPruner'},
 ('LightGBM', 'HyperbandPruner'): {'best_score': 598.7971005905614,
  'best_params': {'n_estimators': 200,
   'learning_rate': 0.15,
   'num_leaves': 31,
   'max_depth': 5,
   'min_child_samples': 1,
   'subsample': 0.5,
   'colsample_bytree': 1.0,
   'reg_alpha': 0.01,
   'reg_lambda': 1,
   'min_child_weight': 0.001,
   'bagging_freq': 1,
   'objective': 'regression',
   'random_state': 42,
   'n_jobs': -1,
   'verbose': -1},
  'test_mse': 598.7971005905614,
  'test_rmse': 24.470331027400537,
  'test_corr_coef': 0.9562046162750685,
  'pruner': 'HyperbandPruner'},
 ('LightGBM', 'ThresholdPruner'): {'best_score': 673.6967643304616,
  'best_params': {'n_estimators': 400,
   'learning_rate': 0.1,
   'num_leaves': 63,
   'max_depth': 3,
   'min_child_samples': 1,
   'subsample': 0.5,
   'colsample_bytree': 1.0,
   'reg_alpha': 0,
   'reg_lambda': 10,
   'min_child_weight': 1e-05,
   'bagging_freq': 1,
   'objective': 'regression',
   'random_state': 42,
   'n_jobs': -1,
   'verbose': -1},
  'test_mse': 673.6967643304616,
  'test_rmse': 25.95566921368936,
  'test_corr_coef': 0.9502168225063722,
  'pruner': 'ThresholdPruner'},
 ('LightGBM', 'WilcoxonPruner'): {'best_score': 727.0376394439177,
  'best_params': {'n_estimators': 500,
   'learning_rate': 0.15,
   'num_leaves': 63,
   'max_depth': 7,
   'min_child_samples': 20,
   'subsample': 1.0,
   'colsample_bytree': 0.7,
   'reg_alpha': 0.01,
   'reg_lambda': 0.1,
   'min_child_weight': 0.01,
   'bagging_freq': 0,
   'objective': 'regression',
   'random_state': 42,
   'n_jobs': -1,
   'verbose': -1},
  'test_mse': 727.0376394439177,
  'test_rmse': 26.96363550124348,
  'test_corr_coef': 0.9387415382908113,
  'pruner': 'WilcoxonPruner'},
 ('GPBoost', 'MedianPruner'): {'best_score': 705.2209234493985,
  'best_params': {'n_estimators': 400,
   'learning_rate': 0.15,
   'max_depth': 5,
   'num_leaves': 31,
   'min_child_samples': 20,
   'subsample': 0.6,
   'colsample_bytree': 0.7,
   'reg_alpha': 0,
   'reg_lambda': 0.5,
   'min_child_weight': 0.001,
   'random_state': 42,
   'n_jobs': -1,
   'verbose': -1},
  'test_mse': 705.2209234493985,
  'test_rmse': 26.555995998067903,
  'test_corr_coef': 0.9409981493259966,
  'pruner': 'MedianPruner'},
 ('GPBoost', 'NopPruner'): {'best_score': 701.3000437709593,
  'best_params': {'n_estimators': 400,
   'learning_rate': 0.15,
   'max_depth': 5,
   'num_leaves': 31,
   'min_child_samples': 20,
   'subsample': 0.7,
   'colsample_bytree': 0.7,
   'reg_alpha': 0.1,
   'reg_lambda': 0.5,
   'min_child_weight': 0.01,
   'random_state': 42,
   'n_jobs': -1,
   'verbose': -1},
  'test_mse': 701.3000437709593,
  'test_rmse': 26.48207023196939,
  'test_corr_coef': 0.9415580579884352,
  'pruner': 'NopPruner'},
 ('GPBoost', 'PatientPruner'): {'best_score': 758.8116327218879,
  'best_params': {'n_estimators': 500,
   'learning_rate': 0.05,
   'max_depth': 7,
   'num_leaves': 63,
   'min_child_samples': 20,
   'subsample': 0.5,
   'colsample_bytree': 0.5,
   'reg_alpha': 1.0,
   'reg_lambda': 0.1,
   'min_child_weight': 0.1,
   'random_state': 42,
   'n_jobs': -1,
   'verbose': -1},
  'test_mse': 758.8116327218879,
  'test_rmse': 27.546535766260845,
  'test_corr_coef': 0.935366184889031,
  'pruner': 'PatientPruner'},
 ('GPBoost', 'PercentilePruner'): {'best_score': 749.6043356730718,
  'best_params': {'n_estimators': 400,
   'learning_rate': 0.05,
   'max_depth': 5,
   'num_leaves': 63,
   'min_child_samples': 20,
   'subsample': 0.5,
   'colsample_bytree': 0.7,
   'reg_alpha': 1.0,
   'reg_lambda': 0,
   'min_child_weight': 0.01,
   'random_state': 42,
   'n_jobs': -1,
   'verbose': -1},
  'test_mse': 749.6043356730718,
  'test_rmse': 27.378903113037087,
  'test_corr_coef': 0.9369245701596264,
  'pruner': 'PercentilePruner'},
 ('GPBoost', 'SuccessiveHalvingPruner'): {'best_score': 715.7122947830827,
  'best_params': {'n_estimators': 500,
   'learning_rate': 0.15,
   'max_depth': 5,
   'num_leaves': 63,
   'min_child_samples': 20,
   'subsample': 0.6,
   'colsample_bytree': 0.7,
   'reg_alpha': 1.0,
   'reg_lambda': 0.5,
   'min_child_weight': 1e-05,
   'random_state': 42,
   'n_jobs': -1,
   'verbose': -1},
  'test_mse': 715.7122947830827,
  'test_rmse': 26.752799755971015,
  'test_corr_coef': 0.9406035271489431,
  'pruner': 'SuccessiveHalvingPruner'},
 ('GPBoost', 'HyperbandPruner'): {'best_score': 729.821365222223,
  'best_params': {'n_estimators': 500,
   'learning_rate': 0.15,
   'max_depth': -1,
   'num_leaves': 31,
   'min_child_samples': 20,
   'subsample': 0.6,
   'colsample_bytree': 0.7,
   'reg_alpha': 0.5,
   'reg_lambda': 0.1,
   'min_child_weight': 0.001,
   'random_state': 42,
   'n_jobs': -1,
   'verbose': -1},
  'test_mse': 729.821365222223,
  'test_rmse': 27.01520618507701,
  'test_corr_coef': 0.9371427391264351,
  'pruner': 'HyperbandPruner'},
 ('GPBoost', 'ThresholdPruner'): {'best_score': 713.8617694748239,
  'best_params': {'n_estimators': 400,
   'learning_rate': 0.15,
   'max_depth': 5,
   'num_leaves': 31,
   'min_child_samples': 20,
   'subsample': 0.8,
   'colsample_bytree': 0.5,
   'reg_alpha': 0,
   'reg_lambda': 1.0,
   'min_child_weight': 0.01,
   'random_state': 42,
   'n_jobs': -1,
   'verbose': -1},
  'test_mse': 713.8617694748239,
  'test_rmse': 26.71819173287788,
  'test_corr_coef': 0.9389779348832684,
  'pruner': 'ThresholdPruner'},
 ('GPBoost', 'WilcoxonPruner'): {'best_score': 698.1615125042936,
  'best_params': {'n_estimators': 500,
   'learning_rate': 0.15,
   'max_depth': 5,
   'num_leaves': 63,
   'min_child_samples': 20,
   'subsample': 0.8,
   'colsample_bytree': 0.7,
   'reg_alpha': 0.1,
   'reg_lambda': 0.5,
   'min_child_weight': 0.01,
   'random_state': 42,
   'n_jobs': -1,
   'verbose': -1},
  'test_mse': 698.1615125042936,
  'test_rmse': 26.422746119665412,
  'test_corr_coef': 0.9417902712966542,
  'pruner': 'WilcoxonPruner'},
 ('CatBoost', 'MedianPruner'): {'best_score': 624.6051161335615,
  'best_params': {'iterations': 1000,
   'learning_rate': 0.03,
   'depth': 4,
   'l2_leaf_reg': 1,
   'border_count': 32,
   'min_data_in_leaf': 1,
   'rsm': 1.0,
   'bagging_temperature': 1,
   'random_seed': 42,
   'verbose': 0},
  'test_mse': 624.6051161335615,
  'test_rmse': 24.992101074810847,
  'test_corr_coef': 0.9533092119864104,
  'pruner': 'MedianPruner'},
 ('CatBoost', 'NopPruner'): {'best_score': 692.5096873387843,
  'best_params': {'iterations': 1000,
   'learning_rate': 0.03,
   'depth': 4,
   'l2_leaf_reg': 1,
   'border_count': 64,
   'min_data_in_leaf': 10,
   'rsm': 0.8,
   'bagging_temperature': 1,
   'random_seed': 42,
   'verbose': 0},
  'test_mse': 692.5096873387843,
  'test_rmse': 26.315578795435684,
  'test_corr_coef': 0.9481648081163799,
  'pruner': 'NopPruner'},
 ('CatBoost', 'PatientPruner'): {'best_score': 615.3808191312761,
  'best_params': {'iterations': 1000,
   'learning_rate': 0.1,
   'depth': 4,
   'l2_leaf_reg': 1,
   'border_count': 32,
   'min_data_in_leaf': 5,
   'rsm': 0.8,
   'bagging_temperature': 0,
   'random_seed': 42,
   'verbose': 0},
  'test_mse': 615.3808191312761,
  'test_rmse': 24.80687040179144,
  'test_corr_coef': 0.9520973423458917,
  'pruner': 'PatientPruner'},
 ('CatBoost', 'PercentilePruner'): {'best_score': 630.5890782836616,
  'best_params': {'iterations': 500,
   'learning_rate': 0.1,
   'depth': 6,
   'l2_leaf_reg': 1,
   'border_count': 64,
   'min_data_in_leaf': 5,
   'rsm': 1.0,
   'bagging_temperature': 1,
   'random_seed': 42,
   'verbose': 0},
  'test_mse': 630.5890782836616,
  'test_rmse': 25.111532774477578,
  'test_corr_coef': 0.9489559729651481,
  'pruner': 'PercentilePruner'},
 ('CatBoost', 'SuccessiveHalvingPruner'): {'best_score': 607.1987681378401,
  'best_params': {'iterations': 500,
   'learning_rate': 0.05,
   'depth': 6,
   'l2_leaf_reg': 1,
   'border_count': 64,
   'min_data_in_leaf': 1,
   'rsm': 0.6,
   'bagging_temperature': 1,
   'random_seed': 42,
   'verbose': 0},
  'test_mse': 607.1987681378401,
  'test_rmse': 24.64140353425186,
  'test_corr_coef': 0.9532324332878932,
  'pruner': 'SuccessiveHalvingPruner'},
 ('CatBoost', 'HyperbandPruner'): {'best_score': 630.5890782836616,
  'best_params': {'iterations': 500,
   'learning_rate': 0.1,
   'depth': 6,
   'l2_leaf_reg': 1,
   'border_count': 64,
   'min_data_in_leaf': 10,
   'rsm': 1.0,
   'bagging_temperature': 1,
   'random_seed': 42,
   'verbose': 0},
  'test_mse': 630.5890782836616,
  'test_rmse': 25.111532774477578,
  'test_corr_coef': 0.9489559729651481,
  'pruner': 'HyperbandPruner'},
 ('CatBoost', 'ThresholdPruner'): {'best_score': 630.5453803317046,
  'best_params': {'iterations': 1000,
   'learning_rate': 0.1,
   'depth': 6,
   'l2_leaf_reg': 1,
   'border_count': 64,
   'min_data_in_leaf': 5,
   'rsm': 1.0,
   'bagging_temperature': 10,
   'random_seed': 42,
   'verbose': 0},
  'test_mse': 630.5453803317046,
  'test_rmse': 25.11066268205012,
  'test_corr_coef': 0.9489600108379744,
  'pruner': 'ThresholdPruner'},
 ('CatBoost', 'WilcoxonPruner'): {'best_score': 643.9058681801139,
  'best_params': {'iterations': 500,
   'learning_rate': 0.03,
   'depth': 4,
   'l2_leaf_reg': 1,
   'border_count': 32,
   'min_data_in_leaf': 10,
   'rsm': 1.0,
   'bagging_temperature': 10,
   'random_seed': 42,
   'verbose': 0},
  'test_mse': 643.9058681801139,
  'test_rmse': 25.37530035645123,
  'test_corr_coef': 0.9524961757264667,
  'pruner': 'WilcoxonPruner'},
 ('NGBoost', 'MedianPruner'): {'best_score': 764.4986435785781,
  'best_params': {'n_estimators': 1000,
   'learning_rate': 0.05,
   'natural_gradient': True,
   'minibatch_frac': 0.5,
   'col_sample': 0.9,
   'Dist': ngboost.distns.normal.Normal,
   'Score': ngboost.scores.LogScore,
   'random_state': 42,
   'verbose': 0},
  'test_mse': 764.4986435785781,
  'test_rmse': 27.649568596608848,
  'test_corr_coef': 0.947510164485886,
  'pruner': 'MedianPruner'},
 ('NGBoost', 'NopPruner'): {'best_score': 686.5311122296366,
  'best_params': {'n_estimators': 500,
   'learning_rate': 0.01,
   'natural_gradient': True,
   'minibatch_frac': 0.5,
   'col_sample': 0.7,
   'Dist': ngboost.distns.normal.Normal,
   'Score': ngboost.scores.LogScore,
   'random_state': 42,
   'verbose': 0},
  'test_mse': 686.5311122296366,
  'test_rmse': 26.201738725314332,
  'test_corr_coef': 0.9534813869482387,
  'pruner': 'NopPruner'},
 ('NGBoost', 'PatientPruner'): {'best_score': 715.6082889546088,
  'best_params': {'n_estimators': 200,
   'learning_rate': 0.05,
   'natural_gradient': True,
   'minibatch_frac': 0.5,
   'col_sample': 0.5,
   'Dist': ngboost.distns.normal.Normal,
   'Score': ngboost.scores.LogScore,
   'random_state': 42,
   'verbose': 0},
  'test_mse': 715.6082889546088,
  'test_rmse': 26.750855854619097,
  'test_corr_coef': 0.9459334028686258,
  'pruner': 'PatientPruner'},
 ('NGBoost', 'PercentilePruner'): {'best_score': 740.7984013307014,
  'best_params': {'n_estimators': 500,
   'learning_rate': 0.05,
   'natural_gradient': True,
   'minibatch_frac': 0.5,
   'col_sample': 0.9,
   'Dist': ngboost.distns.normal.Normal,
   'Score': ngboost.scores.LogScore,
   'random_state': 42,
   'verbose': 0},
  'test_mse': 740.7984013307014,
  'test_rmse': 27.217611969654893,
  'test_corr_coef': 0.9506774767076946,
  'pruner': 'PercentilePruner'},
 ('NGBoost', 'SuccessiveHalvingPruner'): {'best_score': 638.6888742760665,
  'best_params': {'n_estimators': 500,
   'learning_rate': 0.05,
   'natural_gradient': True,
   'minibatch_frac': 0.5,
   'col_sample': 0.7,
   'Dist': ngboost.distns.normal.Normal,
   'Score': ngboost.scores.LogScore,
   'random_state': 42,
   'verbose': 0},
  'test_mse': 638.6888742760665,
  'test_rmse': 25.272294598553305,
  'test_corr_coef': 0.9572337292952867,
  'pruner': 'SuccessiveHalvingPruner'},
 ('NGBoost', 'HyperbandPruner'): {'best_score': 697.6608917411146,
  'best_params': {'n_estimators': 1000,
   'learning_rate': 0.01,
   'natural_gradient': True,
   'minibatch_frac': 0.5,
   'col_sample': 0.7,
   'Dist': ngboost.distns.normal.Normal,
   'Score': ngboost.scores.LogScore,
   'random_state': 42,
   'verbose': 0},
  'test_mse': 697.6608917411146,
  'test_rmse': 26.41327112913345,
  'test_corr_coef': 0.9529991580886127,
  'pruner': 'HyperbandPruner'},
 ('NGBoost', 'ThresholdPruner'): {'best_score': 821.8842540030902,
  'best_params': {'n_estimators': 1000,
   'learning_rate': 0.03,
   'natural_gradient': True,
   'minibatch_frac': 0.5,
   'col_sample': 0.5,
   'Dist': ngboost.distns.normal.Normal,
   'Score': ngboost.scores.LogScore,
   'random_state': 42,
   'verbose': 0},
  'test_mse': 821.8842540030902,
  'test_rmse': 28.668523749978654,
  'test_corr_coef': 0.9381593157350548,
  'pruner': 'ThresholdPruner'},
 ('NGBoost', 'WilcoxonPruner'): {'best_score': 616.7907109047201,
  'best_params': {'n_estimators': 200,
   'learning_rate': 0.05,
   'natural_gradient': True,
   'minibatch_frac': 0.5,
   'col_sample': 0.5,
   'Dist': ngboost.distns.normal.Normal,
   'Score': ngboost.scores.LogScore,
   'random_state': 42,
   'verbose': 0},
  'test_mse': 616.7907109047201,
  'test_rmse': 24.835271508576668,
  'test_corr_coef': 0.952055022204666,
  'pruner': 'WilcoxonPruner'},
 ('HistGradientBoosting', 'MedianPruner'): {'best_score': 894.3602762161045,
  'best_params': {'learning_rate': 0.1,
   'max_iter': 500,
   'max_depth': 7,
   'min_samples_leaf': 20,
   'max_leaf_nodes': None,
   'l2_regularization': 0.1,
   'max_bins': 255,
   'early_stopping': False,
   'validation_fraction': 0.1,
   'n_iter_no_change': 15,
   'loss': 'squared_error',
   'random_state': 42,
   'verbose': 0},
  'test_mse': 894.3602762161045,
  'test_rmse': 29.90585688817668,
  'test_corr_coef': 0.9273930905581117,
  'pruner': 'MedianPruner'},
 ('HistGradientBoosting', 'NopPruner'): {'best_score': 972.4158187613582,
  'best_params': {'learning_rate': 0.15,
   'max_iter': 400,
   'max_depth': None,
   'min_samples_leaf': 5,
   'max_leaf_nodes': 63,
   'l2_regularization': 0.1,
   'max_bins': 255,
   'early_stopping': True,
   'validation_fraction': 0.2,
   'n_iter_no_change': 15,
   'loss': 'squared_error',
   'random_state': 42,
   'verbose': 0},
  'test_mse': 972.4158187613582,
  'test_rmse': 31.183582519674648,
  'test_corr_coef': 0.9415278475522832,
  'pruner': 'NopPruner'},
 ('HistGradientBoosting', 'PatientPruner'): {'best_score': 895.4234055428822,
  'best_params': {'learning_rate': 0.1,
   'max_iter': 400,
   'max_depth': 7,
   'min_samples_leaf': 20,
   'max_leaf_nodes': 15,
   'l2_regularization': 0.1,
   'max_bins': 255,
   'early_stopping': False,
   'validation_fraction': 0.1,
   'n_iter_no_change': 10,
   'loss': 'squared_error',
   'random_state': 42,
   'verbose': 0},
  'test_mse': 895.4234055428822,
  'test_rmse': 29.92362620978417,
  'test_corr_coef': 0.9270471460627402,
  'pruner': 'PatientPruner'},
 ('HistGradientBoosting',
  'PercentilePruner'): {'best_score': 904.9606565486268, 'best_params': {'learning_rate': 0.1,
   'max_iter': 500,
   'max_depth': 7,
   'min_samples_leaf': 20,
   'max_leaf_nodes': None,
   'l2_regularization': 0.0,
   'max_bins': 255,
   'early_stopping': False,
   'validation_fraction': 0.1,
   'n_iter_no_change': 15,
   'loss': 'squared_error',
   'random_state': 42,
   'verbose': 0}, 'test_mse': 904.9606565486268, 'test_rmse': 30.08256399558766, 'test_corr_coef': 0.926980909291644, 'pruner': 'PercentilePruner'},
 ('HistGradientBoosting',
  'SuccessiveHalvingPruner'): {'best_score': 890.0188862670979, 'best_params': {'learning_rate': 0.1,
   'max_iter': 400,
   'max_depth': None,
   'min_samples_leaf': 20,
   'max_leaf_nodes': 31,
   'l2_regularization': 0.1,
   'max_bins': 255,
   'early_stopping': False,
   'validation_fraction': 0.1,
   'n_iter_no_change': 10,
   'loss': 'squared_error',
   'random_state': 42,
   'verbose': 0}, 'test_mse': 890.0188862670979, 'test_rmse': 29.833184313229083, 'test_corr_coef': 0.9272488132634422, 'pruner': 'SuccessiveHalvingPruner'},
 ('HistGradientBoosting', 'HyperbandPruner'): {'best_score': 915.0251868729446,
  'best_params': {'learning_rate': 0.05,
   'max_iter': 500,
   'max_depth': 5,
   'min_samples_leaf': 20,
   'max_leaf_nodes': 63,
   'l2_regularization': 0.5,
   'max_bins': 255,
   'early_stopping': False,
   'validation_fraction': 0.2,
   'n_iter_no_change': 5,
   'loss': 'squared_error',
   'random_state': 42,
   'verbose': 0},
  'test_mse': 915.0251868729446,
  'test_rmse': 30.24938324781093,
  'test_corr_coef': 0.9240920652279543,
  'pruner': 'HyperbandPruner'},
 ('HistGradientBoosting', 'ThresholdPruner'): {'best_score': 887.852870315111,
  'best_params': {'learning_rate': 0.1,
   'max_iter': 500,
   'max_depth': None,
   'min_samples_leaf': 20,
   'max_leaf_nodes': 15,
   'l2_regularization': 0.1,
   'max_bins': 255,
   'early_stopping': False,
   'validation_fraction': 0.2,
   'n_iter_no_change': 5,
   'loss': 'squared_error',
   'random_state': 42,
   'verbose': 0},
  'test_mse': 887.852870315111,
  'test_rmse': 29.796860074764773,
  'test_corr_coef': 0.9275994694463101,
  'pruner': 'ThresholdPruner'},
 ('HistGradientBoosting', 'WilcoxonPruner'): {'best_score': 896.2660904969946,
  'best_params': {'learning_rate': 0.1,
   'max_iter': 200,
   'max_depth': 7,
   'min_samples_leaf': 20,
   'max_leaf_nodes': 63,
   'l2_regularization': 0.1,
   'max_bins': 255,
   'early_stopping': False,
   'validation_fraction': 0.2,
   'n_iter_no_change': 15,
   'loss': 'squared_error',
   'random_state': 42,
   'verbose': 0},
  'test_mse': 896.2660904969946,
  'test_rmse': 29.937703494038995,
  'test_corr_coef': 0.9275452759804554,
  'pruner': 'WilcoxonPruner'},
 ('PGBM', 'MedianPruner'): {'best_score': 611.1316919306154,
  'best_params': {'n_estimators': 500,
   'learning_rate': 0.15,
   'max_leaves': 24,
   'min_split_gain': 0.0,
   'reg_lambda': 5.0,
   'feature_fraction': 0.7,
   'bagging_fraction': 0.5,
   'tree_correlation': 0.2,
   'min_data_in_leaf': 3,
   'max_bin': 64,
   'distribution': 'laplace'},
  'test_mse': 611.1316919306154,
  'test_rmse': 24.72107788771791,
  'test_corr_coef': 0.9520618885844189,
  'pruner': 'MedianPruner'},
 ('PGBM', 'NopPruner'): {'best_score': 759.3681471811483,
  'best_params': {'n_estimators': 200,
   'learning_rate': 0.15,
   'max_leaves': 22,
   'min_split_gain': 0.1,
   'reg_lambda': 5.0,
   'feature_fraction': 0.9,
   'bagging_fraction': 0.5,
   'tree_correlation': 0.2,
   'min_data_in_leaf': 5,
   'max_bin': 64,
   'distribution': 'studentt'},
  'test_mse': 759.3681471811483,
  'test_rmse': 27.55663526595996,
  'test_corr_coef': 0.9347099879037049,
  'pruner': 'NopPruner'},
 ('PGBM', 'PatientPruner'): {'best_score': 660.2953562214459,
  'best_params': {'n_estimators': 200,
   'learning_rate': 0.15,
   'max_leaves': 51,
   'min_split_gain': 0.0,
   'reg_lambda': 5.0,
   'feature_fraction': 0.7,
   'bagging_fraction': 0.7,
   'tree_correlation': 0.2,
   'min_data_in_leaf': 3,
   'max_bin': 64,
   'distribution': 'studentt'},
  'test_mse': 660.2953562214459,
  'test_rmse': 25.696212877026177,
  'test_corr_coef': 0.9508970849157568,
  'pruner': 'PatientPruner'},
 ('PGBM', 'PercentilePruner'): {'best_score': 704.9798058006301,
  'best_params': {'n_estimators': 300,
   'learning_rate': 0.1,
   'max_leaves': 47,
   'min_split_gain': 0.5,
   'reg_lambda': 5.0,
   'feature_fraction': 0.7,
   'bagging_fraction': 0.7,
   'tree_correlation': 0.2,
   'min_data_in_leaf': 3,
   'max_bin': 64,
   'distribution': 'laplace'},
  'test_mse': 704.9798058006301,
  'test_rmse': 26.551455813205987,
  'test_corr_coef': 0.9485435469940052,
  'pruner': 'PercentilePruner'},
 ('PGBM', 'SuccessiveHalvingPruner'): {'best_score': 743.2787056724476,
  'best_params': {'n_estimators': 200,
   'learning_rate': 0.15,
   'max_leaves': 56,
   'min_split_gain': 0.5,
   'reg_lambda': 0.1,
   'feature_fraction': 0.5,
   'bagging_fraction': 0.5,
   'tree_correlation': 0.1,
   'min_data_in_leaf': 3,
   'max_bin': 128,
   'distribution': 'studentt'},
  'test_mse': 743.2787056724476,
  'test_rmse': 27.26313822127687,
  'test_corr_coef': 0.9474862934724956,
  'pruner': 'SuccessiveHalvingPruner'},
 ('PGBM', 'HyperbandPruner'): {'best_score': 716.314319483747,
  'best_params': {'n_estimators': 500,
   'learning_rate': 0.15,
   'max_leaves': 42,
   'min_split_gain': 0.1,
   'reg_lambda': 5.0,
   'feature_fraction': 0.5,
   'bagging_fraction': 0.7,
   'tree_correlation': 0.2,
   'min_data_in_leaf': 3,
   'max_bin': 64,
   'distribution': 'normal'},
  'test_mse': 716.314319483747,
  'test_rmse': 26.764049011383666,
  'test_corr_coef': 0.9496273048994742,
  'pruner': 'HyperbandPruner'},
 ('PGBM', 'ThresholdPruner'): {'best_score': 614.4068909787804,
  'best_params': {'n_estimators': 500,
   'learning_rate': 0.15,
   'max_leaves': 41,
   'min_split_gain': 0.0,
   'reg_lambda': 5.0,
   'feature_fraction': 0.7,
   'bagging_fraction': 0.5,
   'tree_correlation': 0.3,
   'min_data_in_leaf': 3,
   'max_bin': 64,
   'distribution': 'normal'},
  'test_mse': 614.4068909787804,
  'test_rmse': 24.787232418702587,
  'test_corr_coef': 0.951929563310526,
  'pruner': 'ThresholdPruner'},
 ('PGBM', 'WilcoxonPruner'): {'best_score': 642.213251614053,
  'best_params': {'n_estimators': 500,
   'learning_rate': 0.15,
   'max_leaves': 49,
   'min_split_gain': 0.0,
   'reg_lambda': 1.0,
   'feature_fraction': 0.5,
   'bagging_fraction': 0.5,
   'tree_correlation': 0.0,
   'min_data_in_leaf': 3,
   'max_bin': 64,
   'distribution': 'laplace'},
  'test_mse': 642.213251614053,
  'test_rmse': 25.341926754176626,
  'test_corr_coef': 0.949001337869504,
  'pruner': 'WilcoxonPruner'}}

# **Probabilistic distributions using NGBoost**

In [14]:
# Assuming best_params is obtained correctly
best_params = get_best_model_params(best_scores_autosampler, 'NGBoost')

# Remove any conflicting parameters from best_params
incompatible_keys = ['Dist', 'Score']
for key in incompatible_keys:
    best_params.pop(key, None)

# Initialize NGBoost model with best_params
ngb_model = NGBRegressor(Dist=Normal, **best_params)

# Fit the model
ngb_model.fit(X_train, y_train)

# Predict the distribution
pred_dist = ngb_model.pred_dist(X_test)

# Extract mean and standard deviation
predictions_NGB_df = pd.DataFrame({
    'Mean': pred_dist.loc,
    'StdDev': pred_dist.scale,
    'Actual': y_test.ravel()
})

# Calculate and print the mean squared error
mse = mean_squared_error(y_test, pred_dist.loc)
print(f"Mean Squared Error: {mse:.2f}")

# Print the DataFrame with mean and standard deviation
print(predictions_NGB_df.head())

Mean Squared Error: 689.86
         Mean    StdDev  Actual
0  302.543133  1.350491     315
1  190.791582  2.185576     167
2  445.361336  3.597157     516
3  254.254446  2.173802     144
4  126.862620  1.185943     127


In [15]:
generate_and_save_plots(predictions_NGB_df, y_test, "./drive/MyDrive/WATER QUALITY/WQI_uncertainity/WQI_ml/Probabilistic_Distribution//NGBoost_Prob.xlsx")

Excel file saved at ./drive/MyDrive/WATER QUALITY/WQI_uncertainity/WQI_ml/Probabilistic_Distribution//NGBoost_Prob.xlsx


# **Probabilistic distributions using PGBM**

In [16]:
# Assuming best_params is obtained correctly
best_params = get_best_model_params(best_scores_autosampler, 'PGBM')

# Remove any conflicting parameters from best_params
# Note: Adjust the incompatible keys based on PGBM's requirements
incompatible_keys = ['Dist', 'Score']
for key in incompatible_keys:
    best_params.pop(key, None)

# Initialize PGBM model
pgbm_model = PGBM()

def mseloss_objective(yhat, y, sample_weight=None):
    # Ensure that yhat and y are PyTorch tensors
    if not torch.is_tensor(yhat):
        yhat = torch.from_numpy(np.array(yhat)).float()
    if not torch.is_tensor(y):
        y = torch.from_numpy(np.array(y)).float()
    gradient = yhat - y
    hessian = torch.ones_like(yhat)
    return gradient, hessian

def rmseloss_metric(yhat, y, sample_weight=None):
    # Ensure that yhat and y are PyTorch tensors
    if not torch.is_tensor(yhat):
        yhat = torch.from_numpy(np.array(yhat)).float()
    if not torch.is_tensor(y):
        y = torch.from_numpy(np.array(y)).float()
    loss = torch.sqrt(torch.mean((yhat - y) ** 2))
    return loss

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)
# Fit the model
pgbm_model.train((X_train, y_train), objective=mseloss_objective, metric=rmseloss_metric, params = best_params)

# Predict the distribution
pred_dist = pgbm_model.predict_dist(X_test)

# Extract mean and standard deviation
# Note: PGBM might return predictions in a different format, adjust accordingly
predictions_PGBM_df = pd.DataFrame({
    'Mean': pred_dist.mean(axis=0),
    'StdDev': pred_dist.std(axis=0),
    'Actual': y_test.ravel()
})

# Calculate and print the mean squared error
mse = mean_squared_error(y_test, predictions_PGBM_df['Mean'])
print(f"Mean Squared Error: {mse:.2f}")

# Print the DataFrame with mean and standard deviation
print(predictions_PGBM_df.head())

Training on CPU
Estimator 0/500, Train metric: 75.3859
Estimator 1/500, Train metric: 68.6061
Estimator 2/500, Train metric: 62.5382
Estimator 3/500, Train metric: 57.5813
Estimator 4/500, Train metric: 53.0865
Estimator 5/500, Train metric: 48.9136
Estimator 6/500, Train metric: 45.1505
Estimator 7/500, Train metric: 42.7811
Estimator 8/500, Train metric: 40.5681
Estimator 9/500, Train metric: 38.3200
Estimator 10/500, Train metric: 36.0123
Estimator 11/500, Train metric: 33.4947
Estimator 12/500, Train metric: 31.4008
Estimator 13/500, Train metric: 29.6435
Estimator 14/500, Train metric: 27.8629
Estimator 15/500, Train metric: 25.9768
Estimator 16/500, Train metric: 24.9108
Estimator 17/500, Train metric: 23.8302
Estimator 18/500, Train metric: 23.0561
Estimator 19/500, Train metric: 21.9436
Estimator 20/500, Train metric: 20.9398
Estimator 21/500, Train metric: 19.9707
Estimator 22/500, Train metric: 19.0122
Estimator 23/500, Train metric: 18.2646
Estimator 24/500, Train metric: 17

In [17]:
generate_and_save_plots(predictions_PGBM_df, y_test,"./drive/MyDrive/WATER QUALITY/WQI_uncertainity/WQI_ml/Probabilistic_Distribution/PGBM_Prob.xlsx")

Excel file saved at ./drive/MyDrive/WATER QUALITY/WQI_uncertainity/WQI_ml/Probabilistic_Distribution/PGBM_Prob.xlsx


# **IBUG Installation**

In [18]:
# Clone the repository
!git clone https://github.com/jjbrophy47/ibug.git
%cd ibug

# Build extensions if needed
!python setup.py build_ext --inplace

# Install the package
!pip install .

# Verify installation
try:
    from ibug import IBUGWrapper
    print("IBUGWrapper imported successfully!")
except ImportError as e:
    print("Error importing IBUGWrapper:", e)

Cloning into 'ibug'...
remote: Enumerating objects: 1763, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 1763 (delta 18), reused 15 (delta 4), pack-reused 1726 (from 1)
Receiving objects: 100% (1763/1763), 1.31 MiB | 7.32 MiB/s, done.
Resolving deltas: 100% (967/967), done.
/content/ibug
Compiling ibug/parsers/_tree32.pyx because it changed.
Compiling ibug/parsers/_tree64.pyx because it changed.
[1/2] Cythonizing ibug/parsers/_tree32.pyx
performance hint: ibug/parsers/_tree32.pyx:84:43: Exception check after calling '_test_threshold' will always require the GIL to be acquired. Declare '_test_threshold' as 'noexcept' if you control the definition and you're sure you don't want the function to raise exceptions.
performance hint: ibug/parsers/_tree32.pyx:112:43: Exception check after calling '_test_threshold' will always require the GIL to be acquired. Declare '_test_threshold' as 'noexcept' if you control the definition 

In [19]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)
# y_train = y_train.to_numpy()
# y_val = y_val.to_numpy()

# **Probabilistic distributions using LGBM**

In [20]:
# Assume hyperparameter tuning has been done and best_params are obtained
best_params = get_best_model_params(best_scores_autosampler, 'LightGBM')

# --- Ensure IBUG compatibility ---
best_params['reg_alpha'] = 0
best_params['class_weight'] = None
best_params['boosting_type'] = 'gbdt'

# Initialize and train the LightGBM regressor using the best_params
model = LGBMRegressor(**best_params)
model.fit(X_train, y_train)

# Extend the trained model into a probabilistic predictor using IBUGWrapper
prob_model = IBUGWrapper().fit(model, X_train, y_train, X_val=X_val, y_val=y_val)

# Use the model to predict the probabilistic distribution on the test data.
location, scale = prob_model.pred_dist(X_test)

# Optionally, to also get the k-nearest training indices and their values used in modeling
location2, scale2, train_idxs, train_vals = prob_model.pred_dist(X_test, return_kneighbors=True)

dev = scale**0.5
# Create a DataFrame for clarity
predictions_LGBM_df = pd.DataFrame({
    'Mean': location,
    'StdDev': dev,
    'Actual': y_test,
})

print(predictions_LGBM_df.head())

         Mean    StdDev  Actual
0  309.692216  5.064518     315
1  167.575330  4.410946     167
2  432.911410  7.115736     516
3  226.011766  8.541530     144
4  139.448869  4.529832     127


In [22]:
%cd ..
# run it twice

/


In [24]:
generate_and_save_plots(predictions_LGBM_df, y_test, "./content/drive/MyDrive/WATER QUALITY/WQI_uncertainity/WQI_ml/Probabilistic_Distribution/LGBM_Prob.xlsx")

Excel file saved at ./content/drive/MyDrive/WATER QUALITY/WQI_uncertainity/WQI_ml/Probabilistic_Distribution/LGBM_Prob.xlsx


# **Probabilistic distributions using XGBoost**

In [25]:
# Assume hyperparameter tuning has been done and best_params are obtained
best_params = get_best_model_params(best_scores_autosampler, 'XGBoost')

best_params['reg_alpha'] = 0
best_params['scale_pos_weight'] = 1
best_params['base_score'] = 0.5

# Initialize and train the XGBoost regressor using the best_params
model = XGBRegressor(**best_params)
model.fit(X_train, y_train)

# Extend the trained model into a probabilistic predictor using IBUGWrapper
prob_model = IBUGWrapper().fit(model, X_train, y_train, X_val=X_val, y_val=y_val)

# Use the model to predict the probabilistic distribution on the test data.
# This returns the predicted mean (location) and uncertainty (scale, variance proxy)
location, scale = prob_model.pred_dist(X_test)

# Optionally, to also get the k-nearest training indices and their values used in modeling
location, scale, train_idxs, train_vals = prob_model.pred_dist(X_test, return_kneighbors=True)

# Calculate the standard deviation from the variance (scale)
std_dev = scale ** 0.5

# Create a DataFrame for clarity
predictions_XGB_df = pd.DataFrame({
    'Mean': location,
    'StdDev': std_dev,
    'Actual': y_test
})

print(predictions_XGB_df.head())

         Mean    StdDev  Actual
0  291.971161  5.616562     315
1  159.539810  7.057480     167
2  417.764954  8.235899     516
3  228.646912  8.565522     144
4  137.745834  4.370221     127


In [28]:
generate_and_save_plots(predictions_LGBM_df, y_test, "./content/drive/MyDrive/WATER QUALITY/WQI_uncertainity/WQI_ml/Probabilistic_Distribution/XGB_Prob.xlsx")

Excel file saved at ./content/drive/MyDrive/WATER QUALITY/WQI_uncertainity/WQI_ml/Probabilistic_Distribution/XGB_Prob.xlsx


# **Matrix Evaulation**

In [37]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
folder_path = ""
excel_path = os.path.join(folder_path, "./content/drive/MyDrive/WATER QUALITY/WQI_uncertainity/WQI_ml/Probabilistic_Distribution/Matrix Evaluation.xlsx")
with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
    plot_metrics_and_save_to_excel(writer, predictions_NGB_df, y_test, 'NGBoost')
    plot_metrics_and_save_to_excel(writer, predictions_PGBM_df, y_test, 'PGBM')
    plot_metrics_and_save_to_excel(writer, predictions_LGBM_df, y_test, 'LGBM')
    plot_metrics_and_save_to_excel(writer, predictions_XGB_df, y_test, 'XGBoost')